<a href="https://colab.research.google.com/github/akib1162100/Strativ_assesment/blob/main/weather_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [ ]:
# def get_district_info():


In [25]:
import requests
import json
params_dict = {}
def fetch_data():
  url="https://raw.githubusercontent.com/strativ-dev/technical-screening-test/main/bd-districts.json"
  response=requests.get(url=url)
  data=json.loads(response.content)
  dist_list=data.get('districts')
  for location in dist_list:
      if not (location.get('name',None) and location.get('lat',None) and location.get('long',None)):
        continue
      name = location.get('name')
      latitude = float(location.get('lat'))
      longitude = float(location.get('long'))

      params_dict[name] = {
          "latitude": latitude,
          "longitude": longitude,
          "hourly": "temperature_2m",
          "timezone": "auto",
          "past_days": 0
      }
  return params_dict

dist_time = timeit.timeit(fetch_data, number=1)
print(dist_time)




0.13291073800064623


In [26]:
import pickle
import timeit
def save_dict():
    with open('result_dict.pkl', 'wb') as file:
        pickle.dump(params_dict, file)

# Load result_dict from the file
def load_dict():
  with open('result_dict.pkl', 'rb') as file:
      loaded_dict = pickle.load(file)

# Measure the time it takes to save result_dict
save_time = timeit.timeit(save_dict, number=1)

# Measure the time it takes to load result_dict
load_time = timeit.timeit(load_dict, number=1)

# Print the results
print(f"Time to save result_dict: {save_time:.5f} seconds")
print(f"Time to load result_dict: {load_time:.5f} seconds")
# print(loaded_dict)

Time to save result_dict: 0.00036 seconds
Time to load result_dict: 0.00188 seconds


In [47]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
def data_load():
  # Setup the Open-Meteo API client with cache and retry on error
  cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
  retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
  openmeteo = openmeteo_requests.Client(session = retry_session)

  # Make sure all required weather variables are listed here
  # The order of variables in hourly or daily is important to assign them correctly below
  url = "https://api.open-meteo.com/v1/forecast"
  params = {
    "latitude": 23.7115253,
    "longitude": 90.4111451,
    "hourly": "temperature_2m",
    "timezone": "auto",
    "past_days": 0
  }
  responses = openmeteo.weather_api(url, params=params)

  # Process first location. Add a for-loop for multiple locations or weather models
  response = responses[0]
  hourly = response.Hourly()
  hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

  hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s"),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
  )}
  hourly_data["temperature_2m"] = hourly_temperature_2m

  hourly_dataframe = pd.DataFrame(data = hourly_data)
load_time = timeit.timeit(data_load, number=1)
print(load_time)

0.010251028999846312


In [38]:
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 23.7115253,
	"longitude": 90.4111451,
	"daily": "temperature_2m_max",
	"timezone": "auto",
	"past_days": 7
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 23.75°E 90.375°N
Elevation 15.0 m asl
Timezone b'Asia/Dhaka' b'+06'
Timezone difference to GMT+0 21600 s


In [39]:
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

                  date  temperature_2m_max
0  2023-11-15 18:00:00           27.128500
1  2023-11-16 18:00:00           23.778500
2  2023-11-17 18:00:00           27.878500
3  2023-11-18 18:00:00           30.278500
4  2023-11-19 18:00:00           30.628500
5  2023-11-20 18:00:00           28.678499
6  2023-11-21 18:00:00           28.628500
7  2023-11-22 18:00:00           28.528500
8  2023-11-23 18:00:00           28.478498
9  2023-11-24 18:00:00           29.128500
10 2023-11-25 18:00:00           30.978498
11 2023-11-26 18:00:00           30.378500
12 2023-11-27 18:00:00           28.928499
13 2023-11-28 18:00:00           27.928499


In [41]:
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe.tail(24))

                   date  temperature_2m
144 2023-11-28 18:00:00       20.978498
145 2023-11-28 19:00:00       20.428499
146 2023-11-28 20:00:00       20.028500
147 2023-11-28 21:00:00       19.728498
148 2023-11-28 22:00:00       19.428499
149 2023-11-28 23:00:00       19.228498
150 2023-11-29 00:00:00       19.428499
151 2023-11-29 01:00:00       20.228498
152 2023-11-29 02:00:00       21.378500
153 2023-11-29 03:00:00       22.678499
154 2023-11-29 04:00:00       24.128500
155 2023-11-29 05:00:00       25.678499
156 2023-11-29 06:00:00       26.878500
157 2023-11-29 07:00:00       27.578499
158 2023-11-29 08:00:00       27.928499
159 2023-11-29 09:00:00       27.828499
160 2023-11-29 10:00:00       26.978498
161 2023-11-29 11:00:00       25.678499
162 2023-11-29 12:00:00       24.528500
163 2023-11-29 13:00:00       23.728498
164 2023-11-29 14:00:00       23.028500
165 2023-11-29 15:00:00       22.428499
166 2023-11-29 16:00:00       21.878500
167 2023-11-29 17:00:00       21.428499


In [45]:
data_at_2pm = hourly_dataframe[hourly_dataframe['date'].dt.hour == 14]
data_at_2pm.head(10)

,date,temperature_2m
20,2023-11-23 14:00:00,23.678499
44,2023-11-24 14:00:00,23.578499
68,2023-11-25 14:00:00,23.228498
92,2023-11-26 14:00:00,25.178499
116,2023-11-27 14:00:00,23.728498
140,2023-11-28 14:00:00,24.478498
164,2023-11-29 14:00:00,23.028500
